In [ ]:
# !pip install pyperclip
# !pip install selenium

In [4]:
from selenium import webdriver
import time
import pandas as pd
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
import time
import pyperclip

### driver open


In [ ]:
driver = webdriver.Chrome('chromedriver.exe')  # 윈도우
time.sleep(1)

### driver close

In [ ]:
driver.quit()

In [14]:
with open('I:\\WorkSpace\\mykey\\Naver_pw.txt') as file:
    password = file.read()
with open('I:\\WorkSpace\\mykey\\Naver_ID.txt') as file:
    ID = file.read()

### 로그인

In [8]:
#드라이버 로딩 
driver = webdriver.Chrome('./chromedriver.exe') 

# ##사용할 변수 선언 
# #네이버 로그인 주소 
url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com' 
uid = password 
upw = ID
#네이버 로그인 페이지로 이동 
driver.get(url) 
time.sleep(2) #로딩 대기 

#아이디 입력폼 
tag_id = driver.find_element_by_name('id') 
#패스워드 입력폼 
tag_pw = driver.find_element_by_name('pw')

# id 입력 
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기 
tag_id.click() 
pyperclip.copy(uid) 
tag_id.send_keys(Keys.CONTROL, 'v') 
time.sleep(1) 

# pw 입력 # 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기
tag_pw.click() 
pyperclip.copy(upw) 
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1) 

#로그인 버튼 클릭 
login_btn = driver.find_element_by_id('log.login') 
login_btn.click() 
time.sleep(2)



### 전기차 검색 느영나영

In [6]:
driver.get('https://cafe.naver.com/jejutip')
tag_search = driver.find_element_by_name('query').send_keys('전기차')
driver.find_element_by_name("query").send_keys(Keys.ENTER)
time.sleep(2)
driver.switch_to.frame("cafe_main")

In [ ]:
driver.get('https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=20692649&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=0&search.query=%C0%FC%B1%E2%C2%F7&search.viewtype=title&search.page=1')
time.sleep(2)
driver.switch_to.frame('cafe_main')
driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[1]/td[1]/div[2]/div/a[1]')[0].get_attribute("href")

### URL 주소 불러오기

In [7]:
address = []
page = 0
while page > -1:
    
    if page > 101:
        break
    
    try:
        page += 1
        driver.get(f"https://cafe.naver.com/jejutip?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10456178%26search.media=0%26search.searchdate=all%26userDisplay=15%26search.option=0%26search.sortBy=date%26search.searchBy=0%26search.query=%C0%FC%B1%E2%C2%F7%26search.viewtype=title%26search.page={page}")
        driver.switch_to.frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                break
        except:
            pass
    
        for i in range(1,16):
            address.append(driver.find_elements_by_xpath(f'/html/body/div[1]/div/div[5]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]')[0].get_attribute("href"))
            
    except:
        pass
len(address)
# address[0:5]

C:\Users\pc\AppData\Local\Temp/ipykernel_14364/1259453112.py:12: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')


### 게시글 내용,제목,댓글 불러오기

In [ ]:
driver.get(address[1])
time.sleep(2)
driver.switch_to.frame('cafe_main')
try :
    if driver.find_element_by_class_name("title_text")==None:
        title = '제목이없습니다.'
except:
    pass
title =driver.find_element_by_class_name("title_text").text
contents =driver.find_elements_by_tag_name('p')
content = ''
for i,item in enumerate(contents):
    content += contents[i].text+'\n'
print(title)
print(content)

In [ ]:
titles = []
contentss = []
for element in address:
    try:
        driver.get(element)
        time.sleep(2)
        driver.switch_to.frame('cafe_main')
        try: 
            if driver.find_element_by_class_name("nodata").text == '삭제되었거나 없는 게시글입니다.':
                    break
        except:
                pass
        try :
            if driver.find_element_by_class_name("title_text").text == None:
                title = '제목이없습니다.'
                pass
            else:
                title =driver.find_element_by_class_name("title_text").text
        except:
            pass
        if driver.find_elements_by_tag_name('p')[0].text == None:
            contents =driver.find_elements_by_tag_name('b')
        else:
            contents =driver.find_elements_by_tag_name('p')
        content = ''
        for j,item in enumerate(contents):
            content += contents[j].text+'\n'
        titles.append(title)
        contentss.append(content)
    except:
        pass

In [ ]:
df=pd.DataFrame({"제목":titles ,"내용":contentss,"주소":address})
df.to_csv('제사모전기차크롤링.csv',index=False)
df.head(3)
